In [1]:
%%javascript
require(
        ["notebook/js/outputarea"],
        function (oa) {
            oa.OutputArea.auto_scroll_threshold = -1;
            console.log("Setting auto_scroll_threshold to -1");
        });

<IPython.core.display.Javascript object>

In [2]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from urllib.parse import urlparse, parse_qsl
import matplotlib
#from matplotlib.backends.backend_pdf import FigureCanvasPdf
#matplotlib.backend_bases.register_backend('pdf', FigureCanvasPdf)
matplotlib.use('pgf')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from django.db.models import Q, F, Sum, Count, FloatField, Case, When, Value, Max
import matplotlib.patches as patches
from scipy.sparse import csr_matrix, find

from scoping.models import *
from tmv_app.models import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize
from nltk.stem import SnowballStemmer


import textwrap as tw
from utils.text import *

doc = Doc.objects.get(pk=645877)
doc = Doc.objects.get(pk=536594)
stat = RunStats.objects.get(pk=662)

In [3]:
doc.wtopics = Topic.objects.filter(
    run_id=stat,
    doctopic__doc=doc,
    doctopic__score__gt=stat.dthreshold
).order_by('-doctopic__score')

doc.wtopics

<QuerySet [<Topic: {ecosystem, net, nee}>, <Topic: {climat, chang, impact}>, <Topic: {warm, global, centuri}>, <Topic: {speci, distribut, habitat}>, <Topic: {trend, station, observ}>, <Topic: {manag, research, develop}>, <Topic: {system, oper, convent}>]>

In [4]:
doc.dtopics = DynamicTopic.objects.filter(
    run_id=stat,topicdtopic__topic__in=doc.wtopics,
    topicdtopic__topic__doctopic__doc=doc,
).annotate(
    topic_combination=F('topicdtopic__score')*F('topicdtopic__topic__doctopic__score')
).filter(topic_combination__gt=stat.dthreshold).order_by('-topic_combination')


#bla = doc.dtopics.values('title').aggregate(
#    bla=Sum('topic_combination')
#)

doc.dtopics

<QuerySet [<DynamicTopic: {ecosystem, servic, function}>, <DynamicTopic: {warm, global, centuri}>, <DynamicTopic: {climat, chang, impact}>, <DynamicTopic: {speci, distribut, rang}>, <DynamicTopic: {chang, respons, shift}>, <DynamicTopic: {trend, station, year}>, <DynamicTopic: {system, oper, integr}>, <DynamicTopic: {research, sustain, develop}>]>

In [5]:
df = pd.DataFrame.from_dict(list(doc.dtopics.values(
    'id','title',
    'topicdtopic__topic__title',
    'topicdtopic__topic__id',
    'topic_combination',
    'topicdtopic__score'
))).sort_values('title')
df['scaled_link'] = df['topicdtopic__score']/df['topicdtopic__score'].max()
df.head()

,id,title,topic_combination,topicdtopic__score,topicdtopic__topic__id,topicdtopic__topic__title,scaled_link
4,10732,"{chang, respons, shift}",0.026875,1.278635,122304,"{climat, chang, impact}",0.714868
2,10815,"{climat, chang, impact}",0.031765,1.511291,122304,"{climat, chang, impact}",0.844943
0,10789,"{ecosystem, servic, function}",0.053670,1.561630,122295,"{ecosystem, net, nee}",0.873087
7,10743,"{research, sustain, develop}",0.014311,1.325596,122364,"{manag, research, develop}",0.741123
3,10802,"{speci, distribut, rang}",0.028237,1.633683,122287,"{speci, distribut, habitat}",0.913371


In [6]:
tdf = pd.DataFrame.from_dict(list(doc.wtopics.values(
    'id','title','doctopic__score'
)))

tdf['sum']=tdf['doctopic__score'].sum()
tdf['share']=tdf['doctopic__score']/tdf['sum']*100
tdf['x'] = tdf['share'].cumsum().shift().fillna(0)
tdf['center'] = tdf['x']+tdf['share']/2

tdf.head()

,doctopic__score,id,title,sum,share,x,center
0,0.034368,122295,"{ecosystem, net, nee}",0.125847,27.309526,0.000000,13.654763
1,0.021019,122304,"{climat, chang, impact}",0.125847,16.701755,27.309526,35.660403
2,0.019005,122307,"{warm, global, centuri}",0.125847,15.101969,44.011281,51.562265
3,0.017284,122287,"{speci, distribut, habitat}",0.125847,13.734312,59.113250,65.980406
4,0.013948,122331,"{trend, station, observ}",0.125847,11.082973,72.847562,78.389048


In [7]:
dtdf = df.groupby(['title','id']).sum().sort_values('topic_combination',ascending=False).reset_index()
dtdf['sum']=dtdf['topic_combination'].sum()
dtdf['share']=dtdf['topic_combination']/dtdf['sum']*100
dtdf['x'] = dtdf['share'].cumsum().shift().fillna(0)
dtdf['center'] = dtdf['x']+dtdf['share']/2

dtdf.head()

,title,id,topic_combination,topicdtopic__score,topicdtopic__topic__id,scaled_link,sum,share,x,center
0,"{ecosystem, servic, function}",10789,0.053670,1.561630,122295,0.873087,0.227884,23.551592,0.000000,11.775796
1,"{warm, global, centuri}",10800,0.033994,1.788631,122307,1.000000,0.227884,14.917021,23.551592,31.010103
2,"{climat, chang, impact}",10815,0.031765,1.511291,122304,0.844943,0.227884,13.939202,38.468614,45.438215
3,"{speci, distribut, rang}",10802,0.028237,1.633683,122287,0.913371,0.227884,12.390891,52.407816,58.603261
4,"{chang, respons, shift}",10732,0.026875,1.278635,122304,0.714868,0.227884,11.793329,64.798707,70.695371


In [8]:

count_vectorizer = CountVectorizer(
    max_df=1,
    min_df=0,
    ngram_range=(1,1),
    tokenizer=snowball_stemmer(),
    stop_words=stoplist
)

tfidf = count_vectorizer.fit_transform([doc.content])
vocab = count_vectorizer.get_feature_names()

freqs = find(tfidf)[2]
most_frequent = np.argsort(-freqs)

In [26]:
snowball_stemmer = SnowballStemmer("english")

pgf_with_latex = {
    "text.usetex": True,            # use LaTeX to write all text
    "pgf.rcfonts": False,           # Ignore Matplotlibrc
    "text.latex.unicode": True,
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",
        r"\usepackage{xcolor}"     # xcolor for colours
    ],
    "pgf.texsystem" : "xelatex",
    "figure.figsize": [12,7]
}
matplotlib.rcParams.update(pgf_with_latex)

#plt.rcParams["figure.figsize"] = [12,7]
topics = {}
dtopics = {}

fig, axs = plt.subplots(1,2)


ax = axs[0]

row_2 = 1.5

ax.set_xlim([0,100])
ax.set_ylim([0,row_2+1])

colours = [
    '#8dd3c7','#ffffb3','#bebada',
    '#fb8072','#80b1d3','#fdb462',
    '#b3de69','#fccde5','#d9d9d9',
    '#bc80bd','#ccebc5','#ffed6f'
]

for i, row in tdf.iterrows():
    topics[row['id']] = row
    ax.add_patch(patches.Rectangle(
        (row['x'],0),
        width=row['share'],
        height=1,
        fill=True,
        figure=fig,
        alpha=0.5,
        edgecolor='black'
    ))
    ax.text(
        row['center'],0.9,row['title'],rotation=270,
        ha='center'
    )

for i, row in dtdf.iterrows():
    if i >= len(colours):
        c = None
    else:
        c = colours[i]
    dtopics[row['id']] = row
    dtopics[row['id']]['colour'] = c
    ax.add_patch(patches.Rectangle(
        (row['x'],row_2),
        width=row['share'],
        height=1,
        fill=True,
        figure=fig,
        #alpha=0.5,
        edgecolor='black',
        facecolor=c
    ))   
    if row['share'] > 4:
        ax.text(
            row['center'],row_2 +.9,row['title'],rotation=270,
            ha='center'
        )
        
for i, row in df.iterrows():
    dt = dtopics[row['id']]
    t = topics[row['topicdtopic__topic__id']]
    #ax.plot(
    #    [t['center'],dt['center']],[1,row_2],
    #    alpha=row['scaled_link']+0.5,
    #    linewidth=row['scaled_link']*5,
    #    color=dt['colour']
    #)
    ax.fill(
        [t['x'],t['x']+t['share'],dt['x']+dt['share'],dt['x']],
        [1,1,row_2,row_2],
        alpha=0.5,
        color="grey"
    )
    

ax.axis('off')

ax.text(50,-0.1,"\\textbf{Window topics in $doc_1$}",ha="center",va="center")
ax.text(50,row_2+1.1,"\\textbf{Dynamic topics in document}",ha="center",va="center")
    
ax = axs[1]

doc.wrapped_content = r""

#char width of lines
w = 60

for word in tw.fill(tw.dedent(doc.content.rstrip()),width=w).split(' '):
    wordadd = word
    for topic in doc.dtopics:    
        if snowball_stemmer.stem(word) in topic.top_words:
            dt = dtopics[topic.id]
            wordadd='\\colorbox[HTML]{'+dt['colour'].replace('#','').upper()+'}{'+word+'}'
            #print(repr(word))
    doc.wrapped_content+=wordadd+" "
    
title = tw.fill(tw.dedent(doc.title.rstrip()),width=w)

doc.wrapped_content = '\\textbf{'+title+'} \n\n '+doc.wrapped_content

ax.grid(False)
ax.axis('off')
#ax.text(0.2,1,,va="top")

bbox = dict(
    facecolor='white',
    edgecolor='darkgrey',
    pad=20
)

bbox_2 = dict(
    facecolor='white',
    edgecolor='darkgrey',
    pad=0
)

first_pos = (0,0.9)

textbox = ax.text(first_pos[0],first_pos[1],doc.wrapped_content,va="top", bbox=bbox)

r = fig.canvas.get_renderer()
transf = ax.transData.inverted()

pos_shuffle=0.05

for i in [0,1,2,3]:
    x = first_pos[0]+pos_shuffle*i
    y = first_pos[1]+pos_shuffle*i
    if i > 0:
        dbox = ax.text(
            x,y,doc.wrapped_content,
            va="top", bbox=bbox,zorder=0-i
        )
    dbox = ax.text(
        x-0.05,y+0.02,"$doc_{}$".format(i+1),
    )
    

bbox_extent = textbox.get_window_extent(renderer=r)
bbox_extent.y0 = bbox_extent.y0-30
bbox_coords = bbox_extent.transformed(transf)
print(bbox_coords)

theight=bbox_coords.y0 - 0.1

rows = int(theight*200//10)
cols = 5

row_names = []

cells = []

terms = []

for i in range(rows):
    row = []
    row_names.append("doc")
    for j in range(cols):
        if j==0:
            if i==0:
                row.append("Term\nDoc")
            else:
                row.append("$doc_{}$".format(i))
            terms.append("Term\nDoc")
        else:
            w = most_frequent[j-1]
            if i == 0:
                row.append(vocab[w])
            elif i==1:
                row.append(freqs[w])
            else:
                row.append("...")
    cells.append(row)
    
tt_cells = []

for i in range(rows-1):
    tt_row = []
    for j in range(cols):
        if i ==0:
            freq_subset = most_frequent[:cols-1]
            tt_row = ["Term\nTopic"] + [vocab[w] for w in freq_subset]
        else:
            t = doc.wtopics[i-1]
            if j ==0:
                title = t.title.replace('}','').replace('{','').split(',')
                print(title)
                tt_row.append("{}\n{}\n{}".format(*title))
                #tt_row.append("warm\nglobal")
                #tt_row.append(t.title.replace(',','\n'))
            else:
                w = most_frequent[j-1]
                term = Term.objects.get(title=vocab[w])
                try:
                    tt = round(TopicTerm.objects.get(topic=t,term=term).score,2)
                except:
                    tt = 0
                tt_row.append(tt)
                
    tt_cells.append(tt_row)

print(tt_cells)
    
ax.table(
    cellText=cells,
    #colWidths=[0.3,0.2,0.2,0.2,0.2],
    bbox=[0.6,0,0.55,theight]
)

ax.table(
    cellText=tt_cells,
    colWidths=[0.3,0.2,0.2,0.2,0.2],
    bbox=[-0.1,0,0.6,theight+0.05]
)

ax.text(0.2,-0.05,"\\textbf{Topic Term}",ha="center",va="center")
ax.text(0.875,-0.05,"\\textbf{Doc Term}",ha="center",va="center")





fig.patch.set_facecolor('#f0f0f0')

plt.savefig("test.pdf",facecolor=fig.get_facecolor())
plt.close()   
plt.show()

Bbox(x0=0.0, y0=0.3581225262832406, x1=0.872541816009557, y1=0.9000000000000001)
['ecosystem', ' net', ' nee']
['climat', ' chang', ' impact']
['warm', ' global', ' centuri']
[['Term\nTopic', 'chang', 'ecolog', 'global', 'recent'], ['ecosystem\n net\n nee', 0.18, 0.39, 0.01, 0.03], ['climat\n chang\n impact', 6.33, 0, 0.22, 0.19], ['warm\n global\n centuri', 0, 0, 3.72, 0.31]]


In [10]:
title

['speci', ' distribut', ' habitat']

In [11]:
most_frequent




array([ 5, 13, 20, 30,  7, 31,  9, 14,  0, 27, 28, 29, 32, 34, 26, 35, 36,
       37, 38, 39, 40, 41, 33, 25, 21, 23,  1,  2,  3,  4,  6,  8, 10, 24,
       11, 15, 16, 17, 18, 19, 42, 22, 12, 43])

In [12]:
freq_subset



array([ 5, 13, 20, 30])

In [13]:
vocab[5]

'chang'